<a href="https://colab.research.google.com/github/dongjaeseo/AI_Maths/blob/main/ed5011_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, Input, Normalization
from sklearn.model_selection import train_test_split

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
df = pd.read_csv('/content/drive/MyDrive/ed5011/B0005/data/001_B0005_charge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])

In [5]:
print(df)

     Voltage_measured  Current_measured  Temperature  Current_charger  \
0            3.873017         -0.001201    24.655358            0.000   
1            3.479394         -4.030268    24.666480           -4.036   
2            4.000588          1.512731    24.675394            1.500   
3            4.012395          1.509063    24.693865            1.500   
4            4.019708          1.511318    24.705069            1.500   
..                ...               ...          ...              ...   
784          4.190822         -0.003391    24.539397            0.000   
785          4.191299         -0.000942    24.532498            0.000   
786          4.191388         -0.000351    24.518673            0.000   
787          4.191453         -0.001856    24.514363            0.000   
788          4.191078         -0.002892    24.507040            0.000   

     Voltage_charger      Time  
0              0.003     0.000  
1              1.570     2.532  
2              4.726    

In [6]:
df2 = pd.read_csv('/content/drive/MyDrive/ed5011/B0005/data/002_B0005_discharge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])

In [7]:
print(df2)


     Voltage_measured  Current_measured  Temperature  Current_charger  \
0            4.191492         -0.004902    24.330034          -0.0006   
1            4.190749         -0.001478    24.325993          -0.0006   
2            3.974871         -2.012528    24.389085          -1.9982   
3            3.951717         -2.013979    24.544752          -1.9982   
4            3.934352         -2.011144    24.731385          -1.9982   
..                ...               ...          ...              ...   
192          3.258705         -0.001513    35.296711          -0.0006   
193          3.264121         -0.001532    35.021738          -0.0006   
194          3.269030         -0.000406    34.754871          -0.0006   
195          3.273207         -0.000388    34.490078          -0.0006   
196          3.277170         -0.006528    34.230853          -0.0006   

     Voltage_charger      Time  
0              0.000     0.000  
1              4.206    16.781  
2              3.062    

In [8]:
df3 = pd.concat([df, df2])

In [9]:
df3

,Voltage_measured,Current_measured,Temperature,Current_charger,Voltage_charger,Time
0,3.873017,-0.001201,24.655358,0.0000,0.003,0.000
1,3.479394,-4.030268,24.666480,-4.0360,1.570,2.532
2,4.000588,1.512731,24.675394,1.5000,4.726,5.500
3,4.012395,1.509063,24.693865,1.5000,4.742,8.344
4,4.019708,1.511318,24.705069,1.5000,4.753,11.125
...,...,...,...,...,...,...
192,3.258705,-0.001513,35.296711,-0.0006,0.000,3608.594
193,3.264121,-0.001532,35.021738,-0.0006,0.000,3628.953
194,3.269030,-0.000406,34.754871,-0.0006,0.000,3649.375
195,3.273207,-0.000388,34.490078,-0.0006,0.000,3669.875


In [ ]:
from tqdm import tqdm
train_x = []
test_x = []

for i in tqdm(range(1, 335)):
    if i % 2 == 1:
        df1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0005/data/{i:03}_B0005_charge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])
    elif i % 2 == 0:
        df2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0005/data/{i:03}_B0005_discharge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])
        df = pd.concat([df1,df2])

        train_x.append(df)
    
for i in tqdm(range(1, 335)):
    if i % 2 == 1:
        df1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0006/data/{i:03}_B0006_charge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])
    elif i % 2 == 0:
        df2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0006/data/{i:03}_B0006_discharge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])
        df = pd.concat([df1,df2])

        train_x.append(df)

for i in tqdm(range(1, 265)):
    if i % 2 == 1:
        df1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0018/data/{i:03}_B0018_charge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])
    elif i % 2 == 0:
        df2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0018/data/{i:03}_B0018_discharge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])
        df = pd.concat([df1,df2])

        train_x.append(df)

for i in tqdm(range(1, 335)):
    if i % 2 == 1:
        df1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0007/data/{i:03}_B0007_charge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])
    elif i % 2 == 0:
        df2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0007/data/{i:03}_B0007_discharge.csv', header = None, names = ['Voltage_measured','Current_measured','Temperature','Current_charger','Voltage_charger','Time'])
        df = pd.concat([df1,df2])

        test_x.append(df)

  6%|▌         | 20/334 [00:05<01:45,  2.98it/s]

In [ ]:
train_x = np.array(train_x)
test_x = np.array(test_x)

train_x.shape

In [ ]:
# Finding the data with maximum length = 4254

max = 0
for data in train_x:
    if len(data) > max:
        max = len(data)

max

In [ ]:
# Pad data

x_train = np.zeros([train_x.shape[0], max, 6])
x_test = np.zeros([test_x.shape[0], max, 6])

for i, dat_x in enumerate(train_x):
    if len(dat_x) <= max:
        num = len(dat_x)
        x_train[i][-num:] = dat_x
    else:
        x_train[i] = dat_x[-max:]

for i, dat_x in enumerate(test_x):
    if len(dat_x) <= max:
        num = len(dat_x)
        x_test[i][-num:] = dat_x
    else:
        x_test[i] = dat_x[-max:]

print(x_train.shape) # (466, 4254, 6)
print(x_test.shape) # (167, 4254, 6)

In [ ]:
# y data
df_y1 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0005/label/label.csv', header = None)
df_y2 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0006/label/label_B0006.csv', header = None)
df_y3 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0018/label/label_B0018.csv', header = None)
df_y4 = pd.read_csv(f'/content/drive/MyDrive/ed5011/B0007/label/label_B0007.csv', header = None)

df_y = pd.concat([df_y1, df_y2, df_y3])
y_train = np.array(df_y)
y_test = np.array(df_y4)

print(y_train.shape) # (466, 1)
print(y_test.shape) # (167, 1)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size = 0.8, random_state= 0, shuffle = True)

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(x_train.shape[1:])))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
with tf.device('/device:GPU:0'):  
    optim = tf.keras.optimizers.Adam(lr = 1e-3, decay = 1e-5)

    model.compile(loss = 'mse', optimizer = optim, metrics = [tf.keras.metrics.RootMeanSquaredError()])
    history = model.fit(x_train, y_train, epochs = 150, batch_size = 20, verbose = 1)

In [ ]:
model.evaluate(x_test, y_test, batch_size = 20)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['root_mean_squared_error'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['rmse', 'mse'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(x_test)
print(y_pred[:10])

print("\n")

y_actual = y_test[:10]
print(y_actual)

In [ ]:
y_pred = model.predict(x_test)

y_pred_print = y_pred / y_test[0]
y_test_print = y_test / y_test[0]

plt.plot(y_pred_print)
plt.plot(y_test_print)

plt.title('LSTM, Epoch 150, batch size 20')
plt.ylabel('SoH')
plt.xlabel('Cycle')
plt.legend(['Prediction Data','Actual Data'], loc = 'upper right')


In [ ]:
y_diff = y_pred - y_test
percent = 0
for i in range(len(y_test)):
    percent += y_diff[i]/y_test[i]*100

percent = percent /len(y_test)
percent